In [ ]:
import importlib
importlib.reload(main)

In [9]:
import torch

torch.cuda.is_available()

False

In [5]:
import yaml
import pathlib
file = pathlib.Path('config/config.yaml')
# with open(file, 'r') as f:
#     config = yaml.safe_load(f)
#     print(config)

with open(file, 'w') as f:
    yaml.dump(params, f, default_flow_style=False)



In [6]:
with open(file, 'r') as f:
    config = yaml.safe_load(f)
    print(config)

{'batch': 0.9, 'cache': False, 'classes': None, 'data': './datasets/data', 'data_root': 'data', 'device': -1, 'epochs': 10, 'exist_ok': False, 'log_level': 'DEBUG', 'model': 'yolo12n.pt', 'model_name': 'yolo12n', 'noload': False, 'patience': 100, 'profile': False, 'resume': False, 'save': True, 'save_period': -1, 'train': True, 'workers': 8}


In [ ]:
# 需求: pip install ultralytics opencv-python
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO("yolov8n.pt")  # 换成你的权重

src = "input_long_video.mp4"
out_path = "output_annotated.mp4"

# 先用 OpenCV 读一些参数（fps, size）
cap = cv2.VideoCapture(src)
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# mp4 写入（mp4v 或使用 ffmpeg 的编码器）
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

# 逐帧推理并写文件（stream=True，内存低）
for result in model.predict(source=src, stream=True, device="cuda", imgsz=640):
    # result.plot() 会返回 numpy 或 PIL 图像（通常为 RGB）
    annotated = result.plot()  

    # 确保是 numpy array, 并转换为 BGR 供 OpenCV 写入
    if hasattr(annotated, "convert"):  # PIL
        annotated = np.array(annotated)
    # result.plot() 通常是 RGB，转换为 BGR
    frame_bgr = cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR)
    writer.write(frame_bgr)

writer.release()
print("Saved:", out_path)